<a href="https://colab.research.google.com/github/assafrbi/mini-project-Digital-Humanities/blob/main/dh_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
from time import sleep
from random import randint
from warnings import warn
from tqdm import tqdm
from imdb import Cinemagoer
import pandas as pd

### Load Data

In [2]:
url = "https://www.imdb.com/search/title/?title_type=feature&countries=il&start=1&ref_=adv_nxt"
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

num_films_text = html_soup.find_all('div', class_ = 'desc')
num_films=re.search('of (\d.+) titles',str(num_films_text[0])).group(1)
num_films=int(num_films.replace(',', ''))
print(num_films)

1471


In [39]:
ia = Cinemagoer()
ids = []
names = []
synopses = []
years = []
imdb_ratings = []
metascores = []
votes = []
for start in tqdm(range(1, 51, 50)):    
    # Make a get request    
    url = "https://www.imdb.com/search/title/?title_type=feature&countries=il&start="+str(start)+"&ref_=adv_nxt"
    response = get(url)

    # Converting the response to Beautiful Soup Object
    content = BeautifulSoup(response.content, 'lxml')
    movie_containers = content.find_all('div', class_ = 'lister-item mode-advanced')
    # Iterating throught the list of movies 
    # Lists to store the scraped data in
    # Extract data from individual movie container
    for container in movie_containers:
    # If the movie has Metascore, then extract:
        if container.find('div', class_ = 'ratings-metascore') is not None:
          id = container.h3.a['href'][9:-1]
          ids.append(id)

          name = container.h3.a.text
          names.append(name)

          movie = ia.get_movie(id)
          synopses.append(movie.get('plot')[0])

          year = container.h3.find('span', class_ = 'lister-item-year').text
          years.append(year)

          imdb = float(container.strong.text)
          imdb_ratings.append(imdb)

          m_score = container.find('span', class_ = 'metascore').text
          metascores.append(int(m_score))

          vote = container.find('span', attrs = {'name':'nv'})['data-value']
          votes.append(int(vote))

100%|██████████| 1/1 [00:42<00:00, 42.97s/it]


In [40]:
df = pd.DataFrame(
    {'id': ids,
     'movie': names,
     'year': years,
     'synopsis': synopses,
     'imdb': imdb_ratings,
     'metascore': metascores,
     'votes': votes
     })

df

,id,movie,year,synopsis,imdb,metascore,votes
0,9100054,The Lost Daughter,(2021),A woman's beach vacation takes a dark turn whe...,6.7,86,59732
1,3605418,Knock Knock,(I) (2015),A devoted father helps two stranded young wome...,4.9,53,93190
2,0090859,Cobra,(1986),A tough-on-crime street cop must protect the o...,5.8,25,72663
3,0090927,The Delta Force,(1986),"After a plane is hijacked by terrorists, The D...",5.6,37,20393
4,1226753,The Debt,(I) (2010),"In 1965, three Mossad Agents cross into East B...",6.8,65,69353
5,6952960,The Kindergarten Teacher,(2018),A kindergarten teacher in New York becomes obs...,6.7,75,15974
6,9733388,Sublet,(2020),A New York Times travel writer comes to Tel Av...,6.9,67,2215
7,1821641,The Congress,(2013),"An aging, out-of-work actress accepts one last...",6.4,63,18702
8,8000718,The Operative,(2019),A woman is recruited by the Mossad to work und...,5.7,47,6291
9,4552524,Jeruzalem,(2015),When a couple of American young adults fly to ...,4.8,45,9929


In [41]:
from google.colab import drive
drive.mount('/content/gdrive')
with open('/content/gdrive/My Drive/movies.txt', 'w') as f:
  f.write(df.to_csv())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Text Preprocessing


### Topic Modeling - Latent Dirichlet Allocation